In [2]:
import msprime
import matplotlib.pyplot as plt
from IPython.display import display, SVG
import numpy as np
from PIL import Image
from scipy import stats
import tskit
import random
import pandas as pd
from scipy.stats import linregress

In [3]:
def store_data(name, data):
    path = f"TaylorLawsonData//{name}.csv"
    df = pd.DataFrame(data)
    df.to_csv(path, index=False, header=False)
    return None

def get_data(name):
    path = f"TaylorLawsonData//{name}.csv"
    df = pd.read_csv(path, header=None)
    return np.array(df)

In [4]:
# generates a demography and tree sequences, outputs genotype matrices and tree sequence data 
# ancestral population A, Bottleneck population B, General population C
def tree_sequence(initial_A, initial_B, initial_C, time_bottleneck, seq_len,recom_rate,
                  mut_rate, B_sample, C_sample, mig_rate, growth_rate, seed_ts=None):
        # create a big population model and then split it 
    demography = msprime.Demography()
    demography.add_population(name = "A", initial_size = initial_A)
    demography.add_population(name = "B", initial_size = initial_B, growth_rate=growth_rate)
    demography.add_population(name = "C", initial_size = initial_C)
    demography.add_population_split(time = time_bottleneck, derived=["B", "C"], 
                                    ancestral = "A")
    demography.set_migration_rate('B', 'C', mig_rate)
    
    ts1 = msprime.sim_ancestry(samples = {"B":B_sample, "C":C_sample}, 
                               recombination_rate= recom_rate, sequence_length = seq_len,
                               demography=demography, random_seed= seed_ts)
    mts1=msprime.mutate(ts1,rate=mut_rate, random_seed=seed_ts)
    X = mts1.genotype_matrix().transpose()

    XB = X[B_sample*2:,:]
    XC = X[:B_sample*2,:]
    
    return X, XB, XC, mts1


#find frequency of each SNP in genotype matrix 
def find_freq(X): 
    N , L = np.shape(X)
    f = [0 for i in range(L)]
    for i in range(L):
        f[i] = np.sum(X[:,i])/N
    return f
    
# creates vector of Bs based on chosen popuolation(XC here), selection coefficient and inclusion probability     
def generate_B(XC, s, inclusion_prob, seed_B=None):
    if seed_B is not None:
        np.random.seed(seed_B)
        random.seed(seed_B)
    N, L = np.shape(XC)
    f = find_freq(XC)
    B = np.zeros(L)
    
    for i in range (L):
        if f[i]==0 or f[i]==1:
            sd = 0
        else:
            sd= np.power((f[i]*(1-f[i])), s)
        B[i] = np.random.normal(loc=0.0, scale =sd**2)
        if random.random()  >= inclusion_prob :
            B[i] = 0
    return B



# input the genotype matrices, B and heritability and it generates phenotypes for the two populations
def phenotype(XB, XC, B, h2):
    #find variance Vg of population C
    N, L = np.shape(XC)
    Y0C = np.zeros(N)
    for i in range(N-1):
        Y0C[i] = np.dot(B,XC[i,:])
    VarY0C = np.var(Y0C)
        
    # use chosen heritability to find environmental noise E
    if VarY0C == 0:
        VarE  = 0.0001
    else:
        VarE = VarY0gen*(1-h2)/h2  
    
    E = np.zeros(N)
    for i in range (N):
        E[i] = np.random.normal(loc = 0, scale = (VarE)**(0.5))
    
    # create phenotype vector for C
    YC = np.zeros(N)
    for i in range (N):
        YC[i] = Y0C[i] + E[i]
 
    # create phenotyoe vector for B
    Y0B = np.zeros(N)
    YB = np.zeros(N)
    for i in range (N):
        Y0B[i] = np.dot(B,XB[i,:])
        YB[i] = Y0B[i] + E[i]
    
    return YB, YC, Y0B, Y0C

# input genotype matrices, effect sizes and chosen heritability in the general population and output heritability in both populations 
def heritability(XB, XC, B, h2):
    N, L = np.shape(XC)
    Y0C = np.zeros(N)
    for i in range(N-1):
        Y0C[i] = np.dot(B,XC[i,:])
    VarY0C = np.var(Y0C)
        
    # use chosen heritability to find variance of phenotype
    if VarY0C == 0:
        VarE  = 0.0001
    else:
        VarE = VarY0C*(1-h2)/h2  
    
    # find variation in genotype for AJ
    Y0B = np.zeros(N)
    for i in range (N):
        Y0B[i] = np.dot(B,XB[i,:])
    VarY0B = np.var(Y0B)
    
    # use previously found Ve to find heritability for AJ
    h2B = VarY0B/(VarE+VarY0B)
    h2C = h2
    return h2B, h2C


# input demography things and output heritability 
def demography_heritability (initial_A, initial_B, initial_C, time_bottleneck, 
                             seq_len, recom_rate, mut_rate, B_sample, C_sample, s, 
                             h2, inclusion_prob, mig_rate,growth_rate, seed_ts=None, seed_B=None):
    X, XB, XC, mts1 = tree_sequence(initial_A, initial_B, initial_C, time_bottleneck, 
                                 seq_len,recom_rate, mut_rate, B_sample, C_sample, mig_rate, growth_rate, seed_ts)
    B = generate_B(XC, s, inclusion_prob, seed_B=None)
    h2B, h2C = heritability(XB, XC, abs(B), h2)
    return h2B, h2C



def minor_allele_freq(f):
    fnew=[0 for i in range(len(f))]
    for i in range (len(f)):
        if f[i]<0.5:
            fnew[i] = f[i]
        else:
            fnew[i] = 1-f[i]
    return fnew


def inference_power (f, B):
    power = [0 for i in range(len(f))]
    for i in range(len(f)):
        power[i] = f[i]*(1-f[i])*(np.power(B[i], 2))
    total_power = sum(power)
    return total_power


def show_corr_mat(X):
    plt.matshow(abs(np.corrcoef(X.transpose())), cmap = 'Reds')
    plt.show()


In [5]:
initial_A = 10_000
initial_B = 1_000
initial_C = 10_000
seq_len = 10000
recom_rate = 0.00001
mut_rate = 0.000001
B_sample = 1000
C_sample = 1000
s=-0.5
h2 = 0.5 #chosen heritability in general population
inclusion_prob = 1
time_bottleneck = 200
mig_rate = 0
growth_rate = 0

# Figure 1 

In [9]:
X, Xgen, XAJ, mts1 = tree_sequence(initial_A, initial_B, initial_C, time_bottleneck, seq_len,recom_rate, mut_rate, B_sample, C_sample,mig_rate,growth_rate, seed_ts =2)
B = generate_B(Xgen, s, inclusion_prob,0)
fgen = minor_allele_freq(find_freq(Xgen))
fAJ = minor_allele_freq(find_freq(XAJ))

In [10]:
fAJ_1 = []
B_AJ_1 = []
for i in range (len(fAJ)):
    if fAJ[i] < 0.05 and abs(B[i]) <1500:
        fAJ_1.append(fAJ[i])
        B_AJ_1.append(B[i])
B_AJ_1 = np.array(B_AJ_1)

fgen_2 = []
B_gen_2 = []
for i in range (len(fgen)):
    if fgen[i] < 0.05 and abs(B[i]) <1500:
        fgen_2.append(fgen[i])
        B_gen_2.append(B[i])
B_gen_2 = np.array(B_gen_2)


In [18]:
store_data("1a_bottleneck_f" , fAJ_1)
store_data("1a_general_f", fgen_2)
store_data('1a_bottleneck_B', B_AJ_1)
store_data("1a_general_B", B_gen_2)

# Figure 2 

## a 

In [19]:
n_2a =500

fig2a_size1 = 1000
fig2a_h1 = [0 for i in range (n_2a)]
for i in range (n_2a):
    fig2a_h1[i] = demography_heritability (initial_A, fig2a_size1, initial_C, time_bottleneck, 
                             seq_len, recom_rate, mut_rate, B_sample, C_sample, s, h2, inclusion_prob, mig_rate, growth_rate)[0]

fig2a_size2 = 8000    
fig2a_h2 = [0 for i in range (n_2a)]
for i in range (n_2a):
    fig2a_h2[i] = demography_heritability (initial_A, fig2a_size2, initial_C, time_bottleneck, 
                             seq_len, recom_rate, mut_rate, B_sample, C_sample, s, 
                             h2, inclusion_prob, mig_rate, growth_rate)[0]    
    

In [20]:
store_data("2a_size1", fig2a_h1)
store_data("2a_size2", fig2a_h2)

## b

In [22]:
n_2b=30
samples = 100

size_2b = np.linspace(500,10000, num=n_2b)
h2_2b = np.zeros((n_2b, samples))

for j in range (samples):
    for i in range (n_2b):
        h2_2b[i,j] = demography_heritability(initial_A, size_2b[i], initial_C, time_bottleneck, seq_len, recom_rate, mut_rate, B_sample, C_sample, s, h2, inclusion_prob, mig_rate, growth_rate)[0]


In [24]:
store_data("2b_h2matrix", h2_2b)

## c

In [32]:
n_2c =500

fig2c_time1 = 50
fig2c_h1 = [0 for i in range (n_2c)]
for i in range (n_2c):
    fig2c_h1[i] = demography_heritability (initial_A, initial_B, initial_C, fig2c_time1, 
                             seq_len, recom_rate, mut_rate, B_sample, C_sample, s, h2, inclusion_prob, mig_rate, growth_rate)[0]

fig2c_time2 = 500    
fig2c_h2 = [0 for i in range (n_2c)]
for i in range (n_2c):
    fig2c_h2[i] = demography_heritability (initial_A, initial_B, initial_C, fig2c_time2, 
                             seq_len, recom_rate, mut_rate, B_sample, C_sample, s, 
                             h2, inclusion_prob, mig_rate, growth_rate)[0]    


In [33]:
store_data("2c_time1", fig2c_h1)
store_data("2c_time2", fig2c_h2)

## d 

In [35]:
fig2d_n=20
fig2d_samples = 200
fig2d_time = np.linspace(50,500, num=fig2d_n)
fig2d_h = np.zeros((fig2d_n, fig2d_samples))

for j in range (fig2d_samples):
    for i in range (fig2d_n):
        fig2d_h[i,j] = demography_heritability(initial_A, initial_B, initial_C, fig2d_time[i], seq_len, recom_rate, mut_rate, B_sample, C_sample, s, h2, inclusion_prob, mig_rate, growth_rate)[0]
        

h_var_2d = [0 for i in range (fig2d_n)]
for i in range (fig2d_n):
    h_var_2d[i] = np.var(fig2d_h[i,:])
    

In [42]:
B_2d = 1000 
bs_sample_var_2d = np.zeros((fig2d_n, B_2d))
var_2d = np.zeros((fig2d_n,B_2d))
# Empty array to store bootstrap variances
for i in range (fig2d_n):
    bs_samples_2d = np.empty((B_2d, fig2d_samples))
    for b in range (B_2d):
        for j in range(fig2d_samples):
            bs_samples_2d[b,j] = random.choice(fig2d_h[i,:])
            
        bs_sample_var_2d[i, b] = np.var(bs_samples_2d[b,:])
    

# Sort the bootstrap sample variances
    var_2d[i,:] = np.sort(bs_sample_var_2d[i,:])

In [43]:
store_data("2d_timevariances", h_var_2d)
store_data("2d_bootstrap", var_2d)

# Figure 3

## a

In [5]:
n_3a=50
samples_3a = 50
time_3a = [50,100,400]
mig_3a = np.linspace(0,1,num=n_3a)
h2_3a = np.zeros((n_3a,3,samples_3a))

for j in range (samples_3a):
    for i in range (n_3a):
        for k in range(3):
            h2_3a[i,k,j] = demography_heritability(initial_A, initial_B, initial_C, time_3a[k], seq_len, recom_rate, mut_rate, B_sample, C_sample, s, h2, inclusion_prob, mig_3a[i], growth_rate)[0]

        

In [10]:
fig3a_h2_time1=np.array(h2_3a[:,0,:])
fig3a_h2_time2=np.array(h2_3a[:,1,:])
fig3a_h2_time3=np.array(h2_3a[:,2,:])

In [12]:
store_data("3a_h2_time1", fig3a_h2_time1)
store_data("3a_h2_time2", fig3a_h2_time2)
store_data("3a_h2_time3", fig3a_h2_time3)

## b

In [13]:
n_3b = 30
alpha_3b = np.linspace(0,0.03, num = n_3b)

samples_3b = 100
size_3b = [1000, 3000, 5000, 9000]
h2_3b = np.zeros((4, n_3b, samples_3b))

for k in range (4):
    for j in range (samples_3b):
        for i in range (n_3b):
            h2_3b[k,i,j] = demography_heritability(initial_A, size_3b[k], initial_C, time_bottleneck, seq_len, recom_rate, mut_rate, B_sample, C_sample, s, h2, inclusion_prob, mig_rate, alpha_3b[i])[0]


In [15]:
store_data("3b_h2_size1", h2_3b[0,:,:])
store_data("3b_h2_size2", h2_3b[1,:,:])
store_data("3b_h2_size3", h2_3b[2,:,:])
store_data("3b_h2_size4", h2_3b[3,:,:])

# c

In [20]:

alpha_3c = [0,0.004,0.008,0.012, 0.015]

n_3c=20
samples_3c = 200
size_3c = np.linspace(500,10000, num=n_3c)
h2_3c = np.zeros((5, n_3c, samples_3c))


for k in range (5):
    for j in range (samples_3c):
        for i in range (n_3c):
            h2_3c[k,i,j] = demography_heritability(initial_A, size_3c[i], initial_C, time_bottleneck, seq_len, recom_rate, mut_rate, B_sample, C_sample, s, h2, inclusion_prob, mig_rate, alpha_3c[k])[0]


In [21]:
store_data("3c_h2_growth1", h2_3c[0,:,:])
store_data("3c_h2_growth2", h2_3c[1,:,:])
store_data("3c_h2_growth3", h2_3c[2,:,:])
store_data("3c_h2_growth4", h2_3c[3,:,:])
store_data("3c_h2_growth5", h2_3c[4,:,:])

# Figure 4

In [26]:
def observed_trait(XB, XC,B, threshold):
    N, L =np.shape(XC)
    
    B_obs = []
    f_obs = []
    # set SNPs that arent above the threshold to 0
    f = find_freq(XB)
    for i in range (L):
        power = f[i]*(1-f[i])*(np.power(B[i], 2))
        if power > threshold:
            B_obs.append(B[i])
            f_obs.append(f[i])
    return B_obs, f_obs

def observed_heritability(XB, XC, B, h2, threshold):
    N, L =np.shape(XC)
    Y0C = np.zeros(N)
    for i in range(N-1):
        Y0C[i] = np.dot(B, XC[i,:])
    VarY0C = np.var(Y0C)
    
    if VarY0C == 0:
        VarE = 0.0001
    else:
        VarE = VarY0C*(1-h2)/h2
        
    # find variation in genotype for AJ
    Y0B = np.zeros(N)
    for i in range (N):
        Y0B[i] = np.dot(B,XB[i,:])
    VarY0B = np.var(Y0B)    
    
    h2B = VarY0B/(VarE+VarY0B)
    
    B_obs = [0 for i in range(L)]
    # set SNPs that arent above the threshold to 0
    f = find_freq(XB)
    for i in range (L):
        power = f[i]*(1-f[i])*(np.power(B[i], 2))
        if power > threshold:
            B_obs[i] = B[i]
         
            
    # find variation in genotype for AJ
    Y0B_obs = np.zeros(N)
    for i in range (N):
        Y0B_obs[i] = np.dot(B_obs,XB[i,:])
    VarY0B_obs = np.var(Y0B_obs)
    
    # use previously found Ve to find heritability for AJ
    h2B_obs = VarY0B_obs/(VarE+VarY0B_obs)

    return h2B, h2B_obs

def power_quantiles(X, B, quantiles):
    f = find_freq(X)
    power = [0 for i in range (len(B))]
    for i in range(len(B)):
        power[i] = f[i]*(1-f[i])*(np.power(B[i], 2))
    thresholds = np.quantile(power, quantiles)
    return thresholds, power

quantiles_4 = [(1-1/(2**n)) for n in range(13)]

def demography_compute_thresholds(initial_A, initial_B, initial_C, time_bottleneck, 
                             seq_len, recom_rate, mut_rate, B_sample, C_sample, s, 
                             h2, inclusion_prob, mig_rate,growth_rate, seed_ts=None, seed_B=None):
    X, XB, XC, mts1 = tree_sequence(initial_A, initial_B, initial_C, time_bottleneck, 
                                 seq_len,recom_rate, mut_rate, B_sample, C_sample, mig_rate, growth_rate, seed_ts)
    B = generate_B(XC, s, inclusion_prob, seed_B=None)
    thresholds, power = power_quantiles(XB , B, quantiles_4)
    return power, thresholds, XB, XC, B


In [27]:
n_4 = 200
power_all_4 = [0 for i in range (n_4)]
thresholds_all_4 = [0 for i in range (n_4)]
XB_all_4 = [0 for i in range (n_4)]
XC_all_4 = [0 for i in range (n_4)]
B_all_4 = [0 for i in range(n_4)]

for i in range (n_4):
    power_all_4[i], thresholds_all_4[i], XB_all_4[i], XC_all_4[i], B_all_4[i] = demography_compute_thresholds(initial_A, initial_B, initial_C, time_bottleneck, 
                             seq_len, recom_rate, mut_rate, B_sample, C_sample, s, 
                             h2, inclusion_prob, mig_rate,growth_rate, seed_ts=None, seed_B=None)

thresholds_4 = np.mean(thresholds_all_4, axis = 0)  


In [28]:
h2B_all_4 = np.zeros((n_4,13))
h2B_obs_all_4 = np.zeros((n_4,13))
for j in range(13):
    for i in range (n_4):
        h2B_all_4[i,j], h2B_obs_all_4[i,j] = observed_heritability(XB_all_4[i], XC_all_4[i], B_all_4[i], h2, thresholds_4[j])

In [29]:
store_data("4_simulated", h2B_all_4)
store_data("4_observed", h2B_obs_all_4)

## a 

In [ ]:
plt.figure(figsize=(12,9))

plt.hist(h2B_all_4[:,7], bins=np.arange(0, 1 +0.02, 0.03), alpha=0.7, label = "real ")
plt.hist(h2B_obs_all_4[:,7], bins=np.arange(0, 1+0.02 , 0.03), alpha = 0.7, label = 'observed')
#plt.xlim(xmin=0, xmax = 1)
plt.axvline(0.5, color = 'black', linewidth =5, label = "Heritability in the General Population")
plt.xlabel("Heritability",fontsize=14)
plt.ylabel("Frequency",fontsize=14)
plt.axvline(np.mean(h2B_all_4[:,7]), linestyle='dashed', linewidth=2, label = 'real mean')
plt.axvline(np.mean(h2B_obs_all_4[:,7]), color = 'darkorange', linestyle='dashed', linewidth=2, label = 'observed mean')
plt.xlabel("Heritability",fontsize=14)
plt.title("Heritability Distribution with Threshold of 1/2^7")
plt.legend(fontsize=12)
plt.show()

## b

In [ ]:
plt.figure(figsize=(12,9))

plt.hist(h2B_all_4[:,9], bins=np.arange(0, 1 +0.02, 0.03), alpha=0.7, label = "real ")
plt.hist(h2B_obs_all_4[:,9], bins=np.arange(0, 1+0.02 , 0.03), alpha = 0.7, label = 'observed')
#plt.xlim(xmin=0, xmax = 1)
plt.axvline(0.5, color = 'black', linewidth =5, label = "Heritability in the General Population")
plt.xlabel("Heritability",fontsize=14)
plt.ylabel("Frequency",fontsize=14)
plt.axvline(np.mean(h2B_all_4[:,9]), linestyle='dashed', linewidth=2, label = 'real mean')
plt.axvline(np.mean(h2B_obs_all_4[:,9]), color = 'darkorange', linestyle='dashed', linewidth=2, label = 'observed mean')
plt.xlabel("Heritability",fontsize=14)
plt.title("Heritability Distribution with Threshold of 1/2^9")
plt.legend(fontsize=12)
plt.show()

## c

In [ ]:
plt.figure(figsize = (10,10))
plt.plot(np.linspace(0,1,10), np.linspace(0,1,10))
plt.scatter(h2B_all_4[:,0], h2B_obs_all_4[:,3], label = 'n = 3')
plt.scatter(h2B_all_4[:,1], h2B_obs_all_4[:,5], label = 'n = 5')
plt.scatter(h2B_all_4[:,2], h2B_obs_all_4[:,7], label = 'n = 7')
plt.scatter(h2B_all_4[:,3], h2B_obs_all_4[:,9], label = 'n = 9')
plt.xlabel("True Heritbaility")
plt.ylabel("Obeserved Heritability")
plt.title("Heritability Values Under Thresholds of 1/(2^n)")
plt.legend()

## d

In [33]:
corr_4d = []
for i in range (13):
    r = np.corrcoef(h2B_all_4[:,i], h2B_obs_all_4[:,i])[0,1]
    corr_4d.append(r)
corr_4d = np.array(corr_4d)
#transformation

z_4d = [(1/2)*np.log((1+corr_4d[i])/(1-corr_4d[i])) for i in range(13)]


In [34]:
stderr_z_4d = 1/np.sqrt(200 -3)
z_low_4d = z_4d- 2*stderr_z_4d
z_high_4d = z_4d + 2*stderr_z_4d
r_high_4d = (np.exp(2*z_high_4d)-1)/(np.exp(2*z_high_4d)+1)
r_low_4d = (np.exp(2*z_low_4d)-1)/(np.exp(2*z_low_4d)+1)

In [35]:
store_data("4d_correlations", corr_4d)
store_data("4d_rlow", r_low_4d)
store_data("4d_rhigh", r_high_4d)

# Supplementary material 1 

In [6]:
n_sm1 =500

sm1_len1 = 10000
sm1_recom1 = 0.00001
sm1_mut1 = 0.000001
sm1_h1 = [0 for i in range (n_sm1)]
for i in range (n_sm1):
    sm1_h1[i] = demography_heritability (initial_A, initial_B, initial_C, time_bottleneck, 
                             sm1_len1, sm1_recom1, sm1_mut1, B_sample, C_sample, s, 
                             h2, inclusion_prob, mig_rate, growth_rate)[0]

    


sm1_len2 = 1000
sm1_recom2 = 0.0001
sm1_mut2 = 0.00001    
sm1_h2 = [0 for i in range (n_sm1)]
for i in range (n_sm1):
    sm1_h2[i] = demography_heritability (initial_A, initial_B, initial_C, time_bottleneck, 
                             sm1_len2, sm1_recom2, sm1_mut2, B_sample, C_sample, s, 
                             h2, inclusion_prob, mig_rate, growth_rate)[0]
    


sm1_len3 = 100
sm1_recom3 = 0.001
sm1_mut3 = 0.0001    
sm1_h3 = [0 for i in range (n_sm1)]
for i in range (n_sm1):
    sm1_h3[i] = demography_heritability (initial_A, initial_B, initial_C, time_bottleneck, 
                             sm§_len3, sm1_recom3, sm1_mut3, B_sample, C_sample, s, 
                             h2, inclusion_prob, mig_rate, growth_rate)[0]
    
sm1_len4 = 10
sm1_recom4 = 0.01
sm1_mut4 = 0.001    
sm1_h4 = [0 for i in range (n_sm1)]
for i in range (n_sm1):
    sm1_h4[i] = demography_heritability (initial_A, initial_B, initial_C, time_bottleneck, 
                             sm1_seq4, sm1_recom4, sm1_mut4, B_sample, C_sample, s, 
                             h2, inclusion_prob, mig_rate, growth_rate)[0]

NameError: name 'sm_len3' is not defined

In [10]:

sm1_len4 = 10
sm1_recom4 = 0.01
sm1_mut4 = 0.001    
sm1_h4 = [0 for i in range (n_sm1)]
for i in range (n_sm1):
    sm1_h4[i] = demography_heritability (initial_A, initial_B, initial_C, time_bottleneck, 
                             sm1_len4, sm1_recom4, sm1_mut4, B_sample, C_sample, s, 
                             h2, inclusion_prob, mig_rate, growth_rate)[0]
    

In [11]:
store_data("sm1_1", sm1_h1)
store_data("sm1_2", sm1_h2)
store_data("sm1_3", sm1_h3)
store_data("sm1_4", sm1_h4)


# Supplementary material 2

In [12]:
n_sm2 = 500


sm2_len1 = 10000
sm2_recom1 = 0.00001
sm2_mut1 = 0.000001

sm2_h1 = [0 for i in range (n_sm2)]
for i in range (n_sm2):
    sm2_h1[i] = demography_heritability (initial_A, initial_B, initial_C, time_bottleneck, 
                             sm2_len1, sm2_recom1, sm2_mut1, B_sample, C_sample, s, 
                                    h2, inclusion_prob, mig_rate, growth_rate)[0]

sm2_len2 = 10000
sm2_recom2 = 0.000000001
sm2_mut2 = 0.000001
sm2_h2 = [0 for i in range (n_sm2)]
for i in range (n_sm2):
    sm2_h2[i] = demography_heritability (initial_A, initial_B, initial_C, time_bottleneck, 
                             sm2_len2, sm2_recom2, sm2_mut2, B_sample, C_sample, s, 
                                       h2, inclusion_prob, mig_rate, growth_rate)[0]
    

In [13]:
store_data("sm2_1", sm2_h1)
store_data("sm2_2", sm2_h2)